In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [56]:
df = pd.read_csv('https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv')
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [73]:
dt = df.copy()

In [81]:
dt.Age.fillna(dt.Age.mean(), inplace=True)
dt.Embarked.fillna('N',inplace=True)
dt.Cabin.fillna('N',inplace=True)
dt['new_Cabin'] = dt.Cabin.str.slice(0,1)
dt.drop('Cabin',axis=1, inplace=True)
dt.drop(['PassengerId','Name','Ticket'], axis = 1, inplace=True)

In [82]:
gender_enc = LabelEncoder()
gender = gender_enc.fit_transform(dt.Sex)
embark_enc = LabelEncoder()
embark = embark_enc.fit_transform(dt.Embarked)
cabin_enc = LabelEncoder()
cabin = cabin_enc.fit_transform(dt.new_Cabin)
pclass_enc = OneHotEncoder()
pclass = pclass_enc.fit_transform(dt.Pclass.values.reshape(-1,1)).toarray()

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [83]:
dt.drop(['Sex','Embarked','new_Cabin','Pclass'], axis = 1, inplace = True)

In [84]:
dt['gender'] = gender
dt['embark'] = embark
dt['cabin'] = cabin

In [86]:
sc = StandardScaler()
age = sc.fit_transform(dt.Age.values.reshape(-1,1))
adult = np.where(df.Name.str.contains('Mr','Miss'),1,0)

In [87]:
age_enc = LabelEncoder()
age = age_enc.fit_transform(pd.cut(dt.Age,6))
age_oenc = OneHotEncoder()
age = age_oenc.fit_transform(age.reshape(-1,1)).toarray()

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [88]:
enc_data = np.concatenate([gender.reshape(-1,1), embark.reshape(-1,1), cabin.reshape(-1,1), pclass,adult.reshape(-1,1),age], axis = 1)

In [93]:
feature_col = ['SibSp','Parch','Fare']
sur_col = ['Survived']
X = pd.concat([dt[feature_col], pd.DataFrame(enc_data)],axis=1)
y = dt[sur_col]

In [98]:
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [113]:
test_acc =[]
for _ in np.arange(10):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
    dt_tree = DecisionTreeClassifier()
    dt_tree.fit(X_train, y_train)
    acc = accuracy_score(dt_tree.predict(X_test),y_test)
  #  dt_svc = SVC(C=0.3, kernel = 'linear')
  #  dt_svc.fit(X_train,y_train)
  #  acc = accuracy_score(dt_svc.predict(X_test),y_test)
    test_acc.append(acc)
sum(test_acc)/10

0.8039106145251397